# MTA Subway Turnstile Counts ( Work in Progress)


# Phase 1: Data Wrangling
To get started, I need to scrape the text file data from the [mta website](http://web.mta.info/developers/turnstile.html). After looking over the website and inspecting a couple of files, I noted some challenges to keep in mind when collecting and organizing the data:

  + there are two different data types - before and after 10/18/14
      * the older files have multiple entries per line, which will need to be separated
      * the older files are also missing a few fields (station, linename, division)
          - this information seems to be available in separate file, although not entirely complete
      * the older files are formatted using mm-dd-yy while the new files are mm/dd/yy
  + there are 52 files per year, which each hold a week's worth of data
  + to make working with the data more manageable, use sqlite with python
      * this will also make it easier to add in other years if I wanted to expand the analysis
      
As a first pass, I am going to focus only on the year __2013__

## Initializing workspace



In [1]:
import sqlite3 as sql
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd

mta_url = 'http://web.mta.info/developers/turnstile.html'
db_name = 'TURNSTILES.db'
table_name = 'thirteen'
headers = ["CA", "UNIT", "SCP", "STATION", "LINE", "DIVISION", "DATE", "TIME", "DESC", "ENTRIES", "EXITS"]
types = ["TEXT", "TEXT", "TEXT", "TEXT", "TEXT", "TEXT", "DATE", "TIME", "TEXT", "INTEGER", "INTEGER"]

 ## Scrape list of text file links from website and store in dictionary
 
 * Using Requests & BeautifulSoup to get url data and then parse html content
 * I am going to get all of the links currently present, and from there I can filter for the year I want
     + From inspecting the html I noticed that the files I want all of the pattern "data/nyct/turnstile/turnstile" in the links, so I am going to use regular expressions to get only the links that I want 
     + From a ctrl+f I know that there are 301 occurrences of my pattern, so I'm expecting 301 links to be in my dictionary
     + From here, I can either move forward with the full dictionary if I want all the links, or for this purpose I can call another function to create a smaller dictionary of only the links of interest (then I can delete the larger dictionary if I no longer want it)


In [2]:
page_data = requests.get(mta_url)
pg_dat = page_data.content
content = BeautifulSoup(pg_dat, 'html.parser')  
page_links = content.find_all("a", href=True)  

t_links = {}
for link in page_links:  
    if re.match('^data/nyct/turnstile/turnstile.*', link['href']):  
        t_links[link.text] = 'http://web.mta.info/developers/' + link['href']
print len(t_links), 'links in full dictionary'


'''
get only links for 2013
'''


def year_links(url_dict, year):
    y_links = {}
    url_dict = t_links
    for key in url_dict:
        key_year = key.split(',')[2].split()
        if str(year) in key_year:
            y_links[key] = url_dict[key]
    return y_links

year = 2013
y_links = year_links(t_links, year)

print len(y_links), 'links in %d dictionary' % year

301 links in full dictionary
52 links in 2013 dictionary


## Set up database and functions to populate database
* I am going to take advantage of sqlite in python to create a database with a table called 'thirteen' to oranize the date as I read it in
* I know that I need a function to reformat the older files so that they match the newer files and fit into my table structure
  + I need to separate the multiple entries that currently exist on one line of the older files
  + I want to keep the C/A, UNIT, and SCP variables at the beginning of each line, and then repeat those for the other entries 
  + I need to add in a null value for the missing variables: station, linename, and division

In [3]:
'''
set up my database mta.db and create table thirteen
'''


def database_setup(db_name, table_name):

    header_and_types = "id INTEGER PRIMARY KEY AUTOINCREMENT, " + ", ".join([headers[i] + ' ' +
                                                                             types[i] for i in range(len(headers))])
    global conn, cursor
    conn = sql.connect(db_name)
    cursor = conn.cursor()
    create_string = 'CREATE TABLE IF NOT EXISTS '+ table_name + ' (' + header_and_types + ')'
    cursor.execute(create_string)
    return

# run this function to set up database, commit, and then close connection for now
database_setup(db_name, table_name)
conn.commit()
conn.close()

In [4]:
'''
create function to update format of older files - only older files will be passed to this function
'''

def update_format(row):
    
    out_row = []  
    len_id = 3  # chunk of 3 ID variables want to keep & repeat
    len_entry = 5  # chunk the entries within the currently longer row
    cells = row.split(',')  
    
    id_vars = ",".join(cells[0:len_id])

    # match to dataframe to get station, linename, division info (if available) 
    remote = cells[1]
    booth = cells[0] 

    find_station = df.Station[df.Remote == remote][df.Booth == booth].values
    station = find_station[0] if len(find_station) > 0 else 'NULL'

    find_linename = df.Line[df.Remote == remote][df.Booth == booth].values
    linename = find_linename[0] if len(find_linename) > 0 else 'NULL'

    find_division = df.Division[df.Remote == remote][df.Booth == booth].values
    division = find_division[0] if len(find_division) > 0 else 'NULL'
    
    for entry in range(len_id, len(cells), len_entry):
        
        # make date match format of newer files
        date = datetime.strftime(datetime.strptime(cells[entry], "%m-%d-%y"), "%m/%d/%Y")

        added_info = ",".join([station, linename, division, date])
        cleaned_row = id_vars + ',' + added_info + ',' + ','.join(cells[entry + 1: entry + len_entry])
        
        out_row.append(cleaned_row)
        
    return out_row


In [5]:
'''
create function to add entry lines from text files and populate the database
'''

def add_entry(input_line, table_name):

    values = input_line.split(',')
    values = [j.strip() if j != 'NULL' else None for j in values]  # for null values, replace with none
    values.insert(0, None)  # add empty value for id/primary key - auto populate
    value_slots = ",".join(['?']*(len(headers)+1))
    insert_string = 'INSERT INTO ' + table_name + ' VALUES(' + value_slots + ')'

    if len(values) == 12:
        cursor.execute(insert_string, values)



In [10]:
'''
create function to loop through the links in my dictionary and populate the database
'''


def populate_db(url_dict, db_name, table_name, outfile_name):
  
    # read in .csv of station keys for old files
    global df
    df = pd.read_csv('turnstiles_remote_station_key.csv')
    df.columns = ['Remote', 'Booth', 'Station', 'Line', 'Division']

    with open(outfile_name, 'w') as outfile:  # text file to track progress
        
        ctr = 1
        outfile.write('Populating MTA database table turnstiles on' + datetime.now().ctime() + '\n')

        global conn, cursor
        conn = sql.connect(db_name)
        cursor = conn.cursor()


        for key in url_dict:
            url = url_dict[key]

            data = requests.get(url)
            header = next(data.iter_lines()).split(",")
            lines = data.iter_lines()
            
            if len(header) > 11:
                for line in lines:
                    entries = update_format(line)  

                    for entry in entries:
                        add_entry(entry, table_name)  

            else:
                lines.next()
                for line in lines:
                    if len(line) > 1:
                        add_entry(line, table_name)
                        
            outfile.write('populated data for file number ' + str(ctr) + ': ' + url_dict[key] + '\n')
            print 'populated data for file number ' + str(ctr) + ': ' + url_dict[key]
            ctr += 1
            
    



## Execute database population function to scrape, process, and upload text files
   * Call populate function with dictionary of links, database name, table name, and a file name to track which files were entered
       + The populate_db function will output a text file with a statement that the file was uploaded if any lines are uploaded (or really just didn't throw errors). This doesn't necessarily reflect completeness of upload for that file. 
           - Come back to fix this
       + The populate_db does take a non trivial amount of time to execute for a year's worth of data
           - Come back and use trace function to see which steps could be cleaned up to shave off time. 
           - This would be particularly important if I wanted to add more years to my dataframe

In [ ]:
 populate_db(y_links, db_name, table_name, 'downloading_2013_turnstile_data_02062016.txt')

In [9]:
 conn.commit()

# Phase 2: Data Analysis
    
## Challenges and issues to keep in mind:
From initial exploration fo the data I have noticed a few issues that have the potential to create big problems for the analysis (* Note this is only what I've found __so far__* ):

1. The entries and exits values represent an "odometer" so when I want to get the value for any given day, I need to do a subtraction. 
2. Not all station names are unique values, and some stations serve multiple divisions
3. There are a variety of different types of report descriptions - for the sake of simplicity, I am choosing to use only "Regular" reports
4. Unique turnstiles are represented by combinations of Station, Booth, and SCP
    + Ideally, I would have made a primary key for my database that reflected "turnstile id" - since I didn't do this, I'm going to have to group by a number of factors to prevent myself from double counting
5. There is variation in the timing of turnstile reports
    + Stations generally report over fixed intervals, but the timing varies by station. So one station may start reporting at midnight and then proceed every four hours, while another may start at 2 AM
    + Some stations report more frequently than other stations
6. There are instances where a turnstile seems to malfunction or "reset" and it's not always clear why this happened 
    + For instance, for the 08/01/2013 data, a turnstile in the 42 St Bryant Park station appears to reset between 8:00 and 12:00. It's unclear why this happens, and I don't know when this happened within the time interval
    + In other cases, a turnstile may report a very large negative number, and the meaning of this valuue is unclear. For the sake of this analysis, I am excluding negative values 

    

In [2]:
conn = sql.connect(db_name)
cursor = conn.cursor()

### 1. Which station has the most number of turnstile units?

In [ ]:
num_stiles_query = "SELECT COUNT(DISTINCT SCP) as turnstiles, STATION, DIVISION, CA FROM thirteen " \
                   "WHERE STATION is not NULL GROUP BY STATION, DIVISION ORDER BY turnstiles DESC ;"

top_units = pd.read_sql_query(num_stiles_query, conn)

top_units[0:10]

In [ ]:
penn_units = top_units['turnstiles'][top_units['STATION']== '34 ST-PENN STA'].sum()
time_sq_units = pd.concat([top_units['turnstiles'][top_units['STATION'] == '42 ST-TIMES SQ'],
                           top_units['turnstiles'][top_units['STATION'] == '42 ST-PA BUS TE']]).sum()

print 'The Times Square Station complex has {0} turnstiles, and the Penn Station complex has {1} turnstiles '.format(
    *[time_sq_units, penn_units])


### Discussion of answer and approach
* Defining station as distinct station name and division, the __IND__ division of __Penn Station__ has the most (__43__) units.


* Taking into account that __Penn Station__ and __Times Square__ are both stations that serve multiple divisions, which appear in the top 10 counts of units per station, I considered these stations as complex stations and added the turnstile counts across divisions. Additionally, I combined the __Times Sq__ and __Port Authority Bus Terminal__ into one station given that the MTA website considers this one station complex.
    + The __ Times Square complex__ has __89__ turnstiles
    + The __Penn Station complex__ has __83__ turnstiles
    
    
*  __42 St Times Square__ and the __42 St Port Authority__, considered as one station, have the *most* turnstile units

### 2. What was the total number of entries and exits across the subway system for 08/01/2013?

In [ ]:
'''
Query database and organize into dataframes
'''
# query for entry/exit counts for the last time point of the day
end_time_query = "SELECT STATION, DIVISION, CA, SCP, timestamp as ENDTIME, ENTRIES, EXITS from " \
                 "(SELECT MAX(TIME) as timestamp, ENTRIES, EXITS, SCP, CA, DIVISION, STATION from " \
                 "(SELECT * FROM thirteen WHERE  DATE = '08/01/2013' AND DESC = 'REGULAR' AND ENTRIES >= 0 AND " \
                 "EXITS >= 0 GROUP BY STATION, DIVISION, SCP, CA, TIME) GROUP BY STATION, DIVISION, CA, SCP) " \
                 "GROUP BY STATION, DIVISION, CA, SCP;"

# create data frame from query
end_time = pd.read_sql_query(end_time_query, conn)

# query for the entry/exit counts for the first time point of the day
start_time_query = "SELECT STATION, DIVISION, CA, SCP, timestamp as STARTTIME, ENTRIES, " \
                    "EXITS from (SELECT MIN(TIME) as timestamp, " \
                   "ENTRIES, EXITS, SCP, CA, DIVISION, STATION from (SELECT * FROM thirteen WHERE " \
                    "DATE = '08/01/2013' AND " \
                   "DESC = 'REGULAR' AND ENTRIES >= 0 AND EXITS >= 0 GROUP BY STATION, DIVISION, SCP, CA, TIME) " \
                   "GROUP BY STATION, DIVISION, CA, SCP) GROUP BY STATION, DIVISION, CA, SCP;"

# create data frame from query
start_time = pd.read_sql_query(start_time_query, conn)

# define categorical variables that I am going to use later for grouping
start_time['STATION'] = start_time['STATION'].astype('category')
start_time['SCP'] = start_time['SCP'].astype('category')
start_time['DIVISION'] = start_time['DIVISION'].astype('category')

end_time['STATION'] = end_time['STATION'].astype('category')
end_time['SCP'] = end_time['SCP'].astype('category')
end_time['DIVISION'] = end_time['DIVISION'].astype('category')

'''
Define total entries and exits for each day, and deal with "issues" (ie turnstile resets)
'''
entries = end_time['ENTRIES'] - start_time['ENTRIES']

entries[entries < 0] # in fact row 1018 is a problem

# let's check it out
start_time.iloc[1018]
end_time.iloc[1018]

# replace entry total with approximation of daily entries, based on value after reset
entries[1018] = end_time.iloc[1018]['ENTRIES']

# total number of entries on this day
total_entries = sum(entries)

# now do the same for exits

exits = end_time['EXITS'] - start_time['EXITS']
exits[exits < 0] # again 1018 is a problem b/c of the reset

exits[1018] = end_time.iloc[1018]['EXITS']
total_exits = sum(exits)

print 'approximate number of entries on 08/01/2013: ', total_entries

print 'approximate number of exits on 08/01/2013: ', total_exits

print 'approximate total amount of activity on 08/01/2013', total_exits + total_entries

#### Discussion of approach and answer:
* I decided to do two separate queries to get entry/exit counts for the "first" and "last" regular report that day
    + I ruled out negative counts
    + one limitation of defining "first" and "last" based on min/max time stamps is that I could have happened to catch the turnstile at a moment of malfunction, whereas a different time stamp could have been more accurate
    + this approach could have potentially been done in one, more complex, query; however I chose to try to err on the side of interpretability and also to give myself more 'access' to the data to try to deal with flaws
    
    
* I chose to deal with the "reset" turnstile at Bryant Park by assigning that entry/exit count to the end value of the turnstile (after the reset)
    + another approach could have been to replace that turnstile's count with the mean of the other counts at that station on 08/01/2013
    
    
* The number of entries using this approach was __5,043,709__ and the number of exits using this approach was __3,936,543__
    + This gives the total amount of activity as __8,980,252__
    + To check if this order of magnitude was realistic, I decided to quantify a "rider" as one entry and exit. This means that my approach gives an approximation of __4,490,126 riders in one day__, which suggests that the subway system serves __1,638,895,990 riders a year__. This is of the order of magnitude of numbers reported by the MTA website.

### 3. Defining busy-ness as the sume of entry and exit count:

#### Which station was busiest on 08/01/2013?
    

In [ ]:
start_time["BUSY"] = entries + exits # note that the name of the df is no longer meaningful, interested in busy column

# look at the top 10 stations in terms of busy-ness
busy_by_station = start_time.groupby(['STATION', 'DIVISION'])['BUSY'].sum()
busy_by_station.sort_values(ascending = False)[0:10]


Defining station as having a distinct name and division, the busiest station is __Grand Central Station__ with a total turnstile count of __294,257__ turns. However, I know that __Penn Station__ and __Times Square__ stations serve multiple divisions. So I am going to look at the business for both of these stations, combining across divisions (and in the case of Times Square, the other division has a different station name).

In [ ]:
time_sq = pd.concat([start_time[start_time['STATION'] == '42 ST-TIMES SQ'],  start_time[start_time['STATION'] =='42 ST-PA BUS TE']])
time_busy = time_sq['BUSY'].sum()

penn_sq = start_time[start_time['STATION'] == '34 ST-PENN STA']
penn_busy = penn_sq['BUSY'].sum()

print 'Times Square had {0} total turns and Penn Station had {1} total turns'.format(*[time_busy, penn_busy])

If I take into account the fact that these hub stations serve multiple divisions, __Times Square__ is the busiest station with a total of __340,644__ turns. Additionally, Penn Station has __305,771__ turns, which is *greater* than the number of turns for __Grand Central Station__.

#### Which specific turnstile was busiest on 08/01/2013?

In [ ]:
# identify the busiest turnstile
busy_turnstile = start_time[start_time.BUSY == max(start_time.BUSY)]
t_vals_print = [busy_turnstile['CA'].values[0], busy_turnstile['SCP'].values[0],busy_turnstile['STATION'].values[0], 
                busy_turnstile['BUSY'].values[0]]
print 'the busiest turnstile was {0} {1} in station {2} with {3} turns'.format(*t_vals_print) 

The busiest turnstile on 08/01/2013 was __N063a 00-00-00__ in __42 ST Port Authority__ with __10,546__ turns.

### 4. Identify the stations that have seen the most growth and decline in usage in 2013



#### Set Up:
* I created a new table 'daily' of daily entry/exit counts for each station by turnstile
    + This helped to prevent my queries from getting too complex and convoluted
    + I also will need to draw on this information moving forward
    
    
* with this new table, I was able to get a total entry and exit count, as well as a count of busy-ness for each day


In [ ]:
create_daily_table = "CREATE TABLE daily as select * from (SELECT min(end), DATE, entryend, exitend, " \
                        "SCP, CA, DIVISION, STATION from (SELECT MAX(TIME) as end, DATE, "\
                        "ENTRIES as entryend, EXITS as exitend, SCP, CA, DIVISION, STATION from " \
                        "(SELECT * FROM thirteen WHERE  DESC = 'REGULAR' AND ENTRIES >= 0 AND "
                        "EXITS >= 0 AND STATION IS NOT NULL GROUP BY STATION, DIVISION, SCP, CA, DATE, TIME) "
                        "GROUP BY DATE, STATION, DIVISION, CA, SCP " \
                        "UNION " \
                        "SELECT MIN(TIME) as start, DATE, ENTRIES as entrystart, " \
                        "EXITS as exitstart, SCP, CA, DIVISION, STATION from " \
                        "(SELECT * FROM thirteen WHERE  DESC = 'REGULAR' AND ENTRIES >= 0 AND " \
                        "EXITS >= 0 AND STATION IS NOT NULL GROUP BY STATION, DIVISION, SCP, CA, DATE, TIME) "\
                        "GROUP BY DATE, STATION, DIVISION, CA, SCP "\
                        "ORDER BY STATION, SCP) " \
                        "GROUP BY DATE, STATION, DIVISION, SCP ORDER BY STATION, SCP);"

#### Conceptualizing Growth and Decline:
* Given that I have an 'odometer' type reading of turns (busy-ness), I should expect that the turns should increase each day, except when I have a reset or some other disruption. I decided to quantify turns per day, and then look for changes in turns per day. I define __growth as an average positive change in turns per day__, and __decline as an average negative change in turns per day__.


* To quantify turns per day, I used the diff() function in pandas to subtract the previous day's turn count from the current day, for each station. 
    + This method could result in negative or disproportionately large difference in the case of disruption/reset. 
    + I dropped these instances by setting all negative first order differences to NaN. 
    + Also, I know that the busiest stations see turns per day in the hundreds of thousands range, and so any count of turns per day significantly higher than this is likely noise. I set a liberal threshold of 800K for turns per day, and any counts exceeding this threshold were dropped (set to NaN).


* To quantify __change in turns per day__, I used the diff() function again to quantify the fluctuations in rates of turns per day. I then took the average of this quantity for each station.
    + If a station experiences a constant rate of turns per month, this quantity should be about zero
    + if the rate of turns per month goes down, signaling __decline__ in usage, this quantity will be negative
    + If the rate of turns per month increases, signaling __growth__ in usage, this quantity will be positive

In [ ]:
'''
query, pull in data frame, format categories
'''
all_days_query = "SELECT STATION, DIVISION, DATE, sum(entryend) as T_ENTRIES, sum(exitend) as T_EXITS, " \
                 "sum(entryend) + sum(exitend) as BUSY from daily GROUP BY DATE, STATION, DIVISION " \
                 "ORDER BY STATION, DIVISION ;"

days_df = pd.read_sql_query(all_days_query, conn)

days_df['STATION'] = days_df['STATION'].astype('category')
days_df['DIVISION'] = days_df['DIVISION'].astype('category')

'''
take difference between days of 'busy' to get turns per day - grouped by station/division
'''
days_df['DIFF'] = days_df.groupby(['STATION', 'DIVISION'])['BUSY'].diff()

# drop noisy counts (resets, disturbances, etc)
days_df['DIFF'] = days_df['DIFF'].where(days_df['DIFF'] >= 0, 'NaN').astype('float')
days_df['DIFF'] = days_df['DIFF'].where(days_df['DIFF'] < 800000, 'NaN').astype('float')

'''
sanity check: are my busiest stations reporting the highest number of turns per day?
'''
avg_diff = days_df.groupby(['STATION', 'DIVISION'])['DIFF'].mean()
avg_diff.sort_values(ascending = False)[0:10]


'''
get change in turns per day, and average change - grouped by station/division
'''

days_df['DIFF2'] = days_df.groupby(['STATION', 'DIVISION'])['DIFF'].diff()

avg_2nd_diff = days_df.groupby(['STATION', 'DIVISION'])['DIFF2'].mean()

# which stations are declining?
avg_2nd_diff.sort_values()[0:5]

#growing?
avg_2nd_diff.sort_values(ascending = False)[0:5]

'''# look at Z's'''

z_2nd_vals = (avg_2nd_diff - avg_2nd_diff.mean())/avg_2nd_diff.std()
z_2nd_vals.sort_values()[0:5]




